## MLflow's Model Registry

In [20]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [21]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/2', creation_time=1748471083578, experiment_id='2', last_update_time=1748471083578, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1', creation_time=1748283763739, experiment_id='1', last_update_time=1748283763739, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748283452086, experiment_id='0', last_update_time=1748283452086, lifecycle_stage='active', name='Default', tags={}>]

In [23]:
client.create_experiment(name="my-cool-experiment-2")

'3'

Let's check the latest versions for the experiment with id `1`...

In [24]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [25]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: a3c4a353043d4d31974e5c37b644dbfd, rmse: 6.3440
run id: 883dfef543634d6bba6ee3e343d8c488, rmse: 6.3621
run id: 89a14ef1bb7440dd8a2fdb98bdf34e72, rmse: 6.3637
run id: c6c9c55747ec4bc69a2b6742fc83a7b6, rmse: 6.3807
run id: 23c80668c971467498a09f2488acff19, rmse: 6.3851


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [26]:
run_id = "a3c4a353043d4d31974e5c37b644dbfd"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748471722472, current_stage='None', description=None, last_updated_timestamp=1748471722472, name='nyc-taxi-regressor', run_id='a3c4a353043d4d31974e5c37b644dbfd', run_link=None, source='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1/a3c4a353043d4d31974e5c37b644dbfd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [27]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1748471178108, current_stage='None', description=None, last_updated_timestamp=1748471178108, name='nyc-taxi-regressor', run_id='a3c4a353043d4d31974e5c37b644dbfd', run_link=None, source='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1/a3c4a353043d4d31974e5c37b644dbfd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1748470645769, current_stage='None', description='', last_updated_timestamp=1748470645769, name='ny-taxi-regressor', run_id='222ab993adbf4405aaa6aef355c10875', run_link='', source='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1/222ab993adbf4405aaa6aef355c10875/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>]

In [28]:
model_name = "nyc-taxi-regressor"
latest_versions = client.search_model_versions()

for version in latest_versions:
    print(f"version: {version.version}, tags: {version.aliases}")

version: 2, tags: []
version: 1, tags: []
version: 1, tags: []


In [52]:
from datetime import datetime
date = datetime.today().date()

model_version = 1
new_alias = f"stage_{date}"
client.set_registered_model_alias(
    name=model_name,
    version=model_version,
    alias=new_alias
)

In [54]:
model_name = "ny-taxi-regressor"
latest_versions = client.search_model_versions()

for version in latest_versions:
    print(f"version: {version.version}, tags: {version.tags}")

version: 2, tags: {}
version: 1, tags: {}
version: 1, tags: {}


In [55]:
model_version = 1
new_alias = f"production_{date}"
client.set_registered_model_alias(
    name=model_name,
    version=model_version,
    alias=new_alias
)

In [56]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1748471722472, current_stage='None', description=None, last_updated_timestamp=1748471722472, name='nyc-taxi-regressor', run_id='a3c4a353043d4d31974e5c37b644dbfd', run_link=None, source='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1/a3c4a353043d4d31974e5c37b644dbfd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
 <ModelVersion: aliases=[], creation_timestamp=1748471178108, current_stage='None', description=None, last_updated_timestamp=1748471178108, name='nyc-taxi-regressor', run_id='a3c4a353043d4d31974e5c37b644dbfd', run_link=None, source='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1/a3c4a353043d4d31974e5c37b644dbfd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1748470645769, current_stage='None', description='', last_updated_timestamp=1748470645769, name

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [117]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import xgboost as xgb


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, X_test, y_test):
    # model = mlflow.pyfunc.load_model(name)
    xgboost_model = mlflow.xgboost.load_model(name)
    y_pred = xgboost_model.predict(X_test)
    return {"rmse": np.sqrt(mean_squared_error(y_test, y_pred))}

In [108]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [109]:
# Define the run ID
run_id = "222ab993adbf4405aaa6aef355c10875"

client.download_artifacts(run_id=run_id, path="preprocessor/preprocessor.b", dst_path='.')

'/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/preprocessor/preprocessor.b'

In [110]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [111]:
X_test = preprocess(df, dv)
X_test = xgb.DMatrix(X_test)

In [112]:
target = "duration"
y_test = df[target].values

In [113]:
name = f'runs:/{run_id}/models_mlflow'

In [114]:
name

'runs:/222ab993adbf4405aaa6aef355c10875/models_mlflow'

In [118]:
%time test_model(name=name, X_test=X_test, y_test=y_test)

CPU times: user 2.5 s, sys: 5.72 ms, total: 2.51 s
Wall time: 1.54 s


{'rmse': np.float64(6.414347681138584)}